In [1]:
###Reference: https://huggingface.co/blog/fine-tune-wav2vec2-english
import os

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install numpy==1.23.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers==4.17.0
!pip install datasets==1.18.3
!pip install datasets[audio]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [7]:
import torch
import torchaudio
import os
import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor
from datasets import Audio
from datasets import DatasetDict, Dataset

In [8]:
def is_file_exists(directory, filename):
    file_list = os.listdir(directory)
    for file in file_list:
        if file.lower() == filename.lower():
            return True
    return False

In [9]:
directory_wav = "/content/drive/MyDrive/ProcessedWavFiles"
directory_transcript = "/content/drive/MyDrive/ProcessedArpha"
directory_test_wav = "/content/drive/MyDrive/ProcessedWavFilesTest"
directory_test_transcript = "/content/drive/MyDrive/ProcessedArphaTest"
files_with_wav_extension = []
test_files_with_both_extensions = []
files_with_both_extensions = []
test_files_with_wav_extension = []

# Iterate over the directory
for filename in os.listdir(directory_wav):
    if filename.endswith('.wav'):
        files_with_wav_extension.append(filename)

# Check if the corresponding text file exists for each WAV file
for wav_file in files_with_wav_extension:
    txt_file = os.path.splitext(wav_file)[0] + '.txt'
    txt_file_path = os.path.join(directory_transcript, txt_file)
    if is_file_exists(directory_transcript, txt_file):
        files_with_both_extensions.append(wav_file)

# Iterate over the directory
for filename in os.listdir(directory_test_wav):
    if filename.endswith('.wav'):
        test_files_with_wav_extension.append(filename)

# Check if the corresponding test text file exists for each WAV file
for wav_file in test_files_with_wav_extension:
    txt_file = os.path.splitext(wav_file)[0] + '.txt'
    txt_file_path = os.path.join(directory_test_transcript, txt_file)
    if is_file_exists(directory_test_transcript, txt_file):
        test_files_with_both_extensions.append(wav_file)

In [10]:
def open_file_case_insensitive(file_path):
    directory = os.path.dirname(file_path)
    filename = os.path.basename(file_path)
    for file in os.listdir(directory):
        if file.lower() == filename.lower():
            file_path_case_insensitive = os.path.join(directory, file)
            return open(file_path_case_insensitive, "r")
    raise FileNotFoundError(f"File not found: {file_path}")

In [11]:
test_text_list = [] 
test_id_list = [] 
test_audio_list = []
test_file_list = []

for idx, file_name in enumerate(test_files_with_both_extensions):
    txt_file = os.path.splitext(file_name)[0] + '.txt'
    file_path = os.path.join(directory_test_transcript, txt_file)  # Provide the appropriate path to the files
    
    file = open_file_case_insensitive(file_path)
    text = file.read()
    file.close()
    test_text_list.append(text) 
    test_id_list.append(idx)
    waveform, sample_rate = torchaudio.load(os.path.join(directory_test_wav,file_name))
    test_audio_list.append(waveform)
    test_file_list.append(os.path.join(directory_test_wav,file_name))

In [12]:
text_list = [] 
id_list = [] 
audio_list = []
file_list = []

for idx, file_name in enumerate(files_with_both_extensions):
    txt_file = os.path.splitext(file_name)[0] + '.txt'
    file_path = os.path.join(directory_transcript, txt_file)  # Provide the appropriate path to the files
    
    file = open_file_case_insensitive(file_path)
    text = file.read()
    file.close()
    text_list.append(text) 
    id_list.append(idx)
    waveform, sample_rate = torchaudio.load(os.path.join(directory_wav,file_name))
    audio_list.append(waveform)
    file_list.append(os.path.join(directory_wav,file_name))
    print(file_name)

english51.wav
english52.wav
arabic83.wav
english129.wav
romanian7.wav
arabic40.wav
amharic14.wav
spanish38.wav
nepali3.wav
uyghur2.wav
bengali9.wav
arabic41.wav
kurdish2.wav
french14.wav
oriya1.wav
turkish3.wav
mongolian5.wav
armenian7.wav
russian12.wav
mandarin52.wav
english91.wav
turkish2.wav
yapese1.wav
english7.wav
spanish6.wav
mongolian4.wav
spanish39.wav
japanese4.wav
tswana1.wav
korean16.wav
portuguese12.wav
korean6.wav
english102.wav
english6.wav
english115.wav
haitiancreolefrench2.wav
croatian1.wav
turkish1.wav
taiwanese5.wav
english84.wav
russian11.wav
cantonese9.wav
armenian4.wav
armenian5.wav
bai1.wav
arabic82.wav
german26.wav
gusii2.wav
english92.wav
haitiancreolefrench1.wav
german2.wav
english53.wav
haitiancreolefrench3.wav
french28.wav
taiwanese4.wav
poonchi1.wav
nuer1.wav
norwegian7.wav
portuguese38.wav
turkish17.wav
portuguese7.wav
mandarin3.wav
swedish9.wav
faroese1.wav
english101.wav
french17.wav
dutch40.wav
norwegian4.wav
bafang1.wav
nepali1.wav
nepali2.wav
english4

In [13]:
train_dataset = Dataset.from_dict({"audio": file_list, 'text': text_list})
train_dataset = train_dataset.cast_column("audio", Audio())

test_dataset = Dataset.from_dict({"audio": test_file_list, 'text': test_text_list})
test_dataset = test_dataset.cast_column("audio", Audio())

data_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


In [14]:
data_dict

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 1032
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 99
    })
})

In [15]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [16]:
vocabs = data_dict.map(
  extract_all_chars,
  batched=True,
  batch_size=-1,
  keep_in_memory=True, 
  remove_columns=data_dict.column_names["train"]
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))


In [18]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'F': 0,
 'O': 1,
 'D': 2,
 'L': 3,
 'Z': 4,
 'J': 5,
 '1': 6,
 'S': 7,
 'Y': 8,
 'H': 9,
 'C': 10,
 'K': 11,
 'X': 12,
 'I': 13,
 'Q': 14,
 'A': 15,
 'G': 16,
 'R': 17,
 ' ': 18,
 'M': 19,
 '0': 20,
 'P': 21,
 'U': 22,
 'E': 23,
 'V': 24,
 'N': 25,
 'B': 26,
 'T': 27,
 'W': 28}

In [19]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [20]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

31

In [21]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [22]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [23]:
repo_name = "wav2vec2-base-test2"

In [24]:
tokenizer.push_to_hub(repo_name)

To https://huggingface.co/sherryh0112/wav2vec2-base-test2
   cdaed39..916015c  main -> main

   cdaed39..916015c  main -> main



'https://huggingface.co/sherryh0112/wav2vec2-base-test2/commit/916015c59dd9f321ad6967fbd14977991b5f1ed9'

In [25]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [26]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [27]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [28]:
data_dict = data_dict.map(prepare_dataset, remove_columns=data_dict.column_names["train"], num_proc=4)

In [29]:
max_input_length_in_sec = 35.0
data_dict["train"] = data_dict["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])


  0%|          | 0/2 [00:00<?, ?ba/s]

In [30]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [32]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from datasets import load_metric


wer_metric = load_metric("wer")

In [34]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [35]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_q.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you exp

In [36]:
model.freeze_feature_encoder()

In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=5,
  # fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)


In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["test"],
    tokenizer=processor.feature_extractor
)

/content/wav2vec2-base-test2 is already a clone of https://huggingface.co/sherryh0112/wav2vec2-base-test2. Make sure you pull the latest changes with `repo.git_pull()`.


In [39]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 865
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1085


Step,Training Loss,Validation Loss,Wer
500,3.492300,2.956441,0.999567
1000,0.683400,0.454094,0.488604


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 99
  Batch size = 8
Saving model checkpoint to wav2vec2-base-test2/checkpoint-500
Configuration saved in wav2vec2-base-test2/checkpoint-500/config.json
Model weights saved in wav2vec2-base-test2/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-base-test2/checkpoint-500/preprocessor_config.json
Feature extractor saved in wav2vec2-base-test2/preprocessor_config.json
Deleting older checkpoint [wav2vec2-base-test2/checkpoint-100] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely igno

TrainOutput(global_step=1085, training_loss=1.9515870793074506, metrics={'train_runtime': 17517.1691, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.062, 'total_flos': 1.008216521595049e+18, 'train_loss': 1.9515870793074506, 'epoch': 5.0})

In [40]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-base-test2
Configuration saved in wav2vec2-base-test2/config.json
Model weights saved in wav2vec2-base-test2/pytorch_model.bin
Feature extractor saved in wav2vec2-base-test2/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 1.00/360M [00:00<?, ?B/s]

Upload file runs/Jun07_04-31-26_33b28c9d3549/events.out.tfevents.1686112441.33b28c9d3549.67643.0:   0%|       …

To https://huggingface.co/sherryh0112/wav2vec2-base-test2
   83f968e..e6dba77  main -> main

   83f968e..e6dba77  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/sherryh0112/wav2vec2-base-test2
   e6dba77..615f159  main -> main

   e6dba77..615f159  main -> main



'https://huggingface.co/sherryh0112/wav2vec2-base-test2/commit/e6dba77dc542a07f3e4639d8f4baaf47811c4689'

In [42]:
##repo_name = "wav2vec2-base-test2"

In [43]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

loading feature extractor configuration file wav2vec2-base-test2/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

Didn't find file wav2vec2-base-test2/added_tokens.json. We won't load it.
loading file wav2vec2-base-test2/vocab.json
loading file wav2vec2-base-test2/tokenizer_config.json
loading file None
loading file wav2vec2-base-test2/special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file wav2vec2-base-test2/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapt

In [44]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0) #device="cuda"
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [45]:
results = data_dict["test"].map(map_to_result, remove_columns=data_dict["test"].column_names)

0ex [00:00, ?ex/s]

In [58]:
results["pred_str"][:10]

['PLIY1 KAO1L STEH0LAX0 AE1SK HHER1 DXAX1 RIH1N NIY1Z THIH1NGZ WIH1T HHER1R FAX1M NHAX1 STAO1R SIH1KS SPUW1NZ AX1V FREH1SH SNOW0UH0 PIY1Z FAE0IH0V THIH1K SLAE1BZ AX1V BLUW1 CHIY1Z AE1N MEH0IH0BIY0 AX1 SNAE1K FAO1 HHE1 BRAH0DHAX0 BAA1B WIY1 AA0LSOW0 NIY1DX AX1 SMAO1L PLAE0STIH0K STEH0IH0K AX1N AX1 BIH1G TOY1 FRAA1G FAX1 DHAX1 KIH1DZ SHIY1 KAX1N SKUW1P DHIY1Z THIH1NGZ0NTUW0 THRIY1 REH1D BAE1GZ AE1N WIY1 WIH1L GOW0UH MIY1T HHER1 WEH0NZDEH0IH0 E1T DHAX1 TREH0IH0N STEH0IH0SHAX0N',
 'PLIY1Z KAA1L STEH0LAX0 AE1SK HER1 DXAX1 BRIH1NG NHIY1Z THIH1NGZ WIH1TH HHER1R FRAX1M NHAX1 STAO1R SIH1KS SPUW1NZ AX1V FREH1SH SNOW0UH0 PIY1Z FAE0IH0V THIH1K SLAE1BZ AX1V BLUW1 CHIY1Z AE1M MEH0IH0BIY0 AX1 SNAE1K FAX1 HHER1 BRAH0DHER0 BAA1B WIY1 AA0LSOW0 NIY1DX AX1 SMAA1L PLAE0STIH0K SNEH0IH0K AE1N AX1 BIH1G TOY1 FRAA1G FAX1M DHAX1 FO1R DHAX1 KIHAX1DZ SHIY1 KAX1N SKUW1P DHIY1 SHIH1NGZ IH0NTUW0 THRIY1 REH1D BAE1GZ AE1N WIY1 WIH1L GOW0UH0 MIY1T HHER1 WEH0NZDEH0IH0 AE1T DHAX1 TREH0IH0N STEH0IH0SHAX0N',
 'PLIY1Z KAO1L

In [59]:
results['text'][:10]

['PLIY1Z KAO1L STEH0LAH0 AE1SK ER1 DXAX1 BRIH1NG DHIY1Z THIH1NGZ WIH1TH HHER1 FRAX1M DHAX1 STAO0AX0R SIH1KS SPUW1NZ AX1V FREH1SH SNOW0UH0 PIY1Z FAE0IH0V THIH1K SLAE1BZ AX1 BLUW1 CHIY1Z EH1N MEH0IH0BIY0 AX1 SNAE1K FAX1 HHER1 BRAH0DHAX0 BAA1B WIY1 AO0LSAX0 NIY0DAX0 SMAO1L PLAE0STIH0K STEH0IH0K EH0NAX0 BIH1G TOY1 FRAA1G FAX1 DHAX1 KIH1DZ SHIY1 KUH1D SKUW1P DHIY1Z THIH1NGZ TAX1 THRIY1 REH1D BAE1GZ AE1N WIY1 WAX1L GOW0UH0 MIY1T HHER1 WEH0NZEH0IH0 AE1T DAX1 TREH0IH0N STEH0IH0SHAX0N',
 'PLIY1Z KAO1L STEH0LAH0 AE1SK ER1 DXAX1 BRIH1NG DHIY1Z THIH1NGZ WIH1TH HHER1R FRAX1M NAX1 STAO1R SIH1KS SPUW1NZ AX1V FREH1SH SNOW0UH0 PIY1Z FAE0IH0V THIH1K SLAE1BZ AX1V BLAX0UW0 CHIY1Z EH1N MEH0IH0BIY0 AX1 SNAE1K FER1 HHER1 BRAH0DHER0 BAE1B WIY1 AO0LSOW0UH0 NIY1DX AX1 SMAO1L PLAE0STIH0K SNEH0IH0K EH1N AX1 BIH1G TOY1 FRAO1G FRAX1M DHAX1 FAO1R DHAX1 KIH1DZ SHIY1 KAX1N SKUW1P DHIY1Z SIH1NGZ IH0NTAX0 THRIY1 REH1D BAE1GZ EH1N WIY1 WIH1L GOW0UH0 MIY1T HHER1R WIH0NZDEH0IH0 AE1T DHAX1 TREH0IH0N STEH0IH0SHAX0N',
 'PLIY1

In [48]:
len(results["pred_str"])

99

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))